<a href="https://colab.research.google.com/github/constantintonu/IAPD---proiect/blob/main/IAPD_proiect_olx_ro_web_scrapper_pentru_apartamente_de_vanzare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 lxml
!pip install pandas openpyxl



In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

BASE_URL = "https://www.olx.ro"
LISTING_PATH = "/imobiliare/apartamente-garsoniere-de-vanzare/"

# vom salva link-urile unice aici
unique_links = set()
unique_links_partners = set()

# configurarea header-ului

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    )
}

for page in range(1,26):  # paginile 1..100
    params = {
        "currency": "EUR",
        "page": page,
    }

    url = BASE_URL + LISTING_PATH
    print(f"[INFO] Descarc pagina {page}: {url} cu params={params}")

    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
    except Exception as e:
        print(f"[EROARE] Request esuat pentru pagina {page}: {e}")
        continue

    if response.status_code != 200:
        print(f"[AVERTISMENT] Status code {response.status_code} pentru pagina {page}")
        continue

    html = response.text
    with open("resp.html", "w", encoding="utf-8") as f:
      f.write(html);
    soup = BeautifulSoup(html, "lxml")
    body = soup.body

    # 3. gasim toate elementele <div data-cy="l-card">
    cards = body.find_all("div", attrs={"data-cy": "l-card"})
    print(f"  -> Am gasit {len(cards)} card-uri pe pagina {page}")

    olx_count_initial = len(unique_links)
    partenrs_count_initial = len(unique_links_partners)
    for card in cards:
        # 4. in interiorul cardului, cautam DOAR <a class="css-1tqlkj0" href="...">
        a_tags = card.find_all("a", class_="css-1tqlkj0", href=True)

        for a_tag in a_tags:
            #print(f"<a class=css-1tqlkj0>: {a_tag}")
            href = a_tag["href"].strip()

            # 5. transformam in link absolut si il salvam in colectie unica
            full_url = urljoin(BASE_URL, href)
            if(full_url.startswith("https://www.olx.ro/")):
              unique_links.add(full_url)
            else:
              unique_links_partners.add(full_url)

    print(f"  -> Am gasit {len(unique_links) - olx_count_initial} card-uri OLX pe pagina {page}, {len(unique_links_partners) - partenrs_count_initial} card-uri parteneri")
    # pauza mica, sa fim prietenosi cu serverul
    time.sleep(1)

print("\n=== REZULTAT FINAL ===")
print(f"Numar total de anunturi unice gasite: {len(unique_links)}")

# afisam doar primele 20 ca exemplu
for link in list(unique_links)[:20]:
    print(link)

# optional: salvam toate link-urile in fisier
output_file = "olx_apartamente_links.txt"
with open(output_file, "w", encoding="utf-8") as f:
    for link in sorted(unique_links):
        f.write(link + "\n")

output_file = "parteneri_apartamente_links.txt"
with open(output_file, "w", encoding="utf-8") as f:
    for link in sorted(unique_links_partners):
        f.write(link + "\n")

print(f"\nLink-urile au fost salvate in fisierul: {output_file}")


[INFO] Descarc pagina 1: https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/ cu params={'currency': 'EUR', 'page': 1}
  -> Am gasit 52 card-uri pe pagina 1
  -> Am gasit 12 card-uri OLX pe pagina 1, 40 card-uri parteneri
[INFO] Descarc pagina 2: https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/ cu params={'currency': 'EUR', 'page': 2}
  -> Am gasit 52 card-uri pe pagina 2
  -> Am gasit 8 card-uri OLX pe pagina 2, 43 card-uri parteneri
[INFO] Descarc pagina 3: https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/ cu params={'currency': 'EUR', 'page': 3}
  -> Am gasit 52 card-uri pe pagina 3
  -> Am gasit 6 card-uri OLX pe pagina 3, 46 card-uri parteneri
[INFO] Descarc pagina 4: https://www.olx.ro/imobiliare/apartamente-garsoniere-de-vanzare/ cu params={'currency': 'EUR', 'page': 4}
  -> Am gasit 52 card-uri pe pagina 4
  -> Am gasit 7 card-uri OLX pe pagina 4, 40 card-uri parteneri
[INFO] Descarc pagina 5: https://www.olx.ro/imobiliare/apartamente-

In [ ]:
import re
import csv

def extract_ad_details(url):
    """Intoarce un dict cu detalii despre anunt (sau None daca esueaza)."""
    try:
        resp = requests.get(url, headers=headers, timeout=10)
    except Exception as e:
        print(f"[EROARE] Request esuat pentru anunt: {url} -> {e}")
        return None

    if resp.status_code != 200:
        print(f"[AVERTISMENT] Status code {resp.status_code} pentru anunt: {url}")
        return None

    soup = BeautifulSoup(resp.text, "lxml")

    # TITLU
    title = None
    title_tag = soup.find(attrs={"data-cy": "ad_title"})
    if title_tag:
        title = title_tag.get_text(strip=True)
    else:
        h1 = soup.find("h1")
        if h1:
            title = h1.get_text(strip=True)

    # PRET
    price = None
    price_container = soup.find(attrs={"data-testid": "ad-price-container"})
    if price_container:
        price = price_container.get_text(" ", strip=True)
    else:
        price_tag = soup.find(attrs={"data-testid": "ad-price"})
        if price_tag:
            price = price_tag.get_text(" ", strip=True)

    # LOCATIE + DATA
    location_date = None
    loc_tag = soup.find(attrs={"data-testid": "location-date"})
    if loc_tag:
        location_date = loc_tag.get_text(" ", strip=True)

    # SUPRAFATA (m²) – cautam text cu m²
    surface_m2 = None
    text_with_m2 = soup.find(string=re.compile(r"\b\d+\s*(m²|m2)\b"))
    if text_with_m2:
        # extragem doar numarul
        m = re.search(r"(\d+)\s*(m²|m2)", text_with_m2)
        if m:
            surface_m2 = m.group(1)

    return {
        "url": url,
        "title": title,
        "price": price,
        "location_date": location_date,
        "surface_m2": surface_m2,
    }

# Parcurgem TOATE link-urile stranse mai sus
rows = []
print("\n=== INCEP EXTRAGERE DETALII ANUNTURI ===")

for i, ad_url in enumerate(unique_links, start=1):
    print(f"[{i}/{len(unique_links)}] Procesez: {ad_url}")
    data = extract_ad_details(ad_url)
    if data:
        rows.append(data)
    time.sleep(1)  # pauza intre anunturi

print(f"\nAm extras detalii pentru {len(rows)} anunturi.")



=== INCEP EXTRAGERE DETALII ANUNTURI ===
[1/155] Procesez: https://www.olx.ro/d/oferta/vanzare-apartament-cu-3-camere-dec-in-braila-calea-calarasilor-nou-IDjW2E7.html
[2/155] Procesez: https://www.olx.ro/d/oferta/apartament-3-camere-tip-5-palladium-residence-1-IDk1EDn.html
[3/155] Procesez: https://www.olx.ro/d/oferta/apartament-de-vanzare-IDjRfib.html
[4/155] Procesez: https://www.olx.ro/d/oferta/apartament-intabulat-2-camere-decomandat-top-proprietar-imobil-locuit-IDjV3tY.html
[5/155] Procesez: https://www.olx.ro/d/oferta/apartament-3-camere-110mp-utili-baneasa-aviatiei-IDk3dsq.html
[6/155] Procesez: https://www.olx.ro/d/oferta/apartament-3-camere-renovat-nou-etaj-1-mobilat-si-utilat-nou-cluj-IDjUKLI.html
[7/155] Procesez: https://www.olx.ro/d/oferta/vand-apartament-2-camerere-in-craiova-craiovita-promenada-penny-IDjWnpf.html
[8/155] Procesez: https://www.olx.ro/d/oferta/3-camere-mansarda-bragadiru-direct-dezvoltator-pret-fara-tva-IDjxvU9.html
[9/155] Procesez: https://www.olx.ro/d/

In [ ]:
output_csv = "olx_apartamente_detalii.csv"

fieldnames = ["url", "title", "price", "location_date", "surface_m2"]

with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

print(f"Fisierul CSV a fost salvat ca: {output_csv}")


Fisierul CSV a fost salvat ca: olx_apartamente_detalii.csv


Parsare anunt v2

In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    )
}

def fetch_html(url: str) -> str:
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    return resp.text

def parse_storia_details(html: str) -> dict:
    """
    Primește HTML-ul unei pagini de anunț Storia și
    întoarce un dict {eticheta: valoare} pentru toate
    perechile de tip:
      'Suprafață utilă' -> '45 m²',
      'Numărul de camere' -> '2', etc.
    """
    soup = BeautifulSoup(html, "lxml")
    body = soup.body

    if body is None:
        return {}

    # 1. Găsim containerul principal de detalii (AdDetailsBase)
    details_root = body.find("div", attrs={"data-sentry-component": "AdDetailsBase"})
    if details_root is None:
        # fallback: ce ai pus în exemplu
        details_root = body.find("div", attrs={"data-sentry-element": "StyledListContainer"})
    if details_root is None:
        return {}

    details = {}

    # 2. În interior, toate blocurile de tip "ItemGridContainer"
    #    fiecare conține: <div>Label</div> + <div>Valoare</div>
    containers = details_root.select('[data-sentry-element="ItemGridContainer"]')

    for grid in containers:
        # luăm DOAR div-urile directe, ca să nu prindem alte chestii
        children = [
            d for d in grid.find_all("div", recursive=False)
            if d.get_text(strip=True)
        ]

        if len(children) < 2:
            continue

        label = children[0].get_text(" ", strip=True)
        value = children[1].get_text(" ", strip=True)

        # scoatem ":" de la finalul label-ului, dacă există
        if label.endswith(":"):
            label = label[:-1].strip()

        details[label] = value

    return details


Exemplu extragere data dintr-un singur anunt

In [ ]:
url = "https://www.storia.ro/ro/oferta/vanzare-apartament-2-camere-in-ploiesti-zona-nord-cameliei-IDDRz6.html"

html = fetch_html(url)
detalii = parse_storia_details(html)

for k, v in detalii.items():
    print(f"{k}: {v}")


Suprafață utilă: 45 m²
Numărul de camere: 2
Încălzire: fără informații
Etaj: 4/4
Chirie: fără informații
Stare: gata de utilizare
Tip proprietate: locuință nouă
Forma de proprietate: fără informații
Liber de la: fără informații
Tip vânzător: agenție
Informații suplimentare: fără informații
Anul construcției: 1967
Lift: nu
Facilități: aer condiționat


Extragere date din toate anunturile:

In [ ]:
for i, ad_url in enumerate(unique_links_partners, start=1):
    print(f"[{i}/{len(unique_links_partners)}] Procesez: {ad_url}")
    try:
        try:
            html = fetch_html(ad_url)
        except Exception as e:
            print(f"    ⚠ fetch_html a eșuat, retry 1x...")
            html = fetch_html(ad_url)  # retry once

        details = parse_storia_details(html)

        if details:
            details["url"] = ad_url
            print(f"anunt {i}: {details}")
            all_ads_data.append(details)

    except Exception as e:
        print(f"  -> ❌ Ignor eroarea și continui: {e}")
        continue


[1/1135] Procesez: https://www.storia.ro/ro/oferta/apartament-1-camer-zona-centrul-vechi-IDA2Cw.html
anunt 1: {'Suprafață utilă': '31 m²', 'Numărul de camere': '1', 'Încălzire': 'fără informații', 'Etaj': '2', 'Chirie': 'fără informații', 'Stare': 'fără informații', 'Tip proprietate': 'locuință utilizată', 'Forma de proprietate': 'fără informații', 'Liber de la': 'fără informații', 'Tip vânzător': 'agenție', 'Informații suplimentare': 'fără informații', 'Anul construcției': '2022', 'Lift': 'nu', 'Facilități': 'mobilier', 'Siguranță': 'supraveghere video', 'url': 'https://www.storia.ro/ro/oferta/apartament-1-camer-zona-centrul-vechi-IDA2Cw.html'}
[2/1135] Procesez: https://www.storia.ro/ro/oferta/locuinta-moderna-cu-o-camera-mutare-rapida-sos-salaj-rahova-IDEQcK.html
anunt 2: {'Suprafață utilă': '38 m²', 'Numărul de camere': '1', 'Încălzire': 'centrală pe gaz', 'Etaj': '5/6', 'Chirie': 'fără informații', 'Stare': 'gata de utilizare', 'Tip proprietate': 'locuință utilizată', 'Forma de pr

KeyboardInterrupt: 

In [ ]:
!pip install pandas openpyxl


In [ ]:
import re
import pandas as pd
df = pd.DataFrame(all_ads_data)
print(df.head())


  Suprafață utilă Numărul de camere        Încălzire      Etaj  \
0           31 m²                 1  fără informații         2   
1           38 m²                 1  centrală pe gaz       5/6   
2         68.5 m²                 3  centrală pe gaz       3/3   
3           52 m²                 2  centrală pe gaz  parter/4   
4          119 m²                 4  fără informații       3/3   

            Chirie              Stare     Tip proprietate  \
0  fără informații    fără informații  locuință utilizată   
1  fără informații  gata de utilizare  locuință utilizată   
2  fără informații  gata de utilizare  locuință utilizată   
3  fără informații  gata de utilizare  locuință utilizată   
4  fără informații    fără informații  locuință utilizată   

   Forma de proprietate      Liber de la Tip vânzător  \
0       fără informații  fără informații      agenție   
1  drept de proprietate       2025-11-27      agenție   
2  drept de proprietate  fără informații      agenție   
3  drept

In [ ]:
def parse_surface_m2(value):
    """
    Primește un string gen '45 m²' sau '45,5 m²'
    și întoarce un număr (float sau int).
    """
    if not isinstance(value, str):
        return None

    # căutăm prima apariție de număr (ex: 45, 45.5, 45,5)
    m = re.search(r'(\d+[.,]?\d*)', value)
    if not m:
        return None

    num_str = m.group(1).replace(",", ".")
    try:
        val = float(num_str)
        # dacă vrei întreg, poți converti la int:
        return int(round(val))
    except ValueError:
        return None

df["Suprafata_utila_m2"] = df["Suprafață utilă"].apply(parse_surface_m2)


In [ ]:
output_csv = "storia_apartamente_normalizat.csv"
output_xlsx = "storia_apartamente_normalizat.xlsx"

# CSV (cu BOM pentru diacritice ok în Excel pe Windows)
df.to_csv(output_csv, index=False, encoding="utf-8-sig")

# Excel
df.to_excel(output_xlsx, index=False)

print("Fișiere salvate:")
print(" -", output_csv)
print(" -", output_xlsx)


Fișiere salvate:
 - storia_apartamente_normalizat.csv
 - storia_apartamente_normalizat.xlsx
